In [1]:
import pandas as pd
import json
import numpy as np
import re

In [2]:
# datos_json = []

# with open('./Data/metadata-sitios/1.json', 'r') as archivo:
#     for linea in archivo:
#         try:
#             objeto_json = json.loads(linea)
#             datos_json.append(objeto_json)
#         except json.JSONDecodeError as e:
#             print(f"Error al cargar la línea: {e}")

# df = pd.DataFrame(datos_json)

In [25]:
# df.to_parquet('1.parquet',index=False)

In [166]:
df = pd.read_parquet('./parquet_google/11.parquet')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              275000 non-null  object 
 1   address           266492 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       16466 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          273104 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             17193 non-null   object 
 10  hours             197196 non-null  object 
 11  MISC              205018 non-null  object 
 12  state             200910 non-null  object 
 13  relative_results  244159 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


In [49]:
df.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,Pittsburgh Circus School,"Pittsburgh Circus School, 6800 Brighton Rd 2nd...",0x8834f5bb828394c3:0x8c8cd12a81fcc81a,None,40.505382,-80.078735,"[Gymnastics center, Circus]",3.9,16,None,"[[Saturday, 9AM–1PM], [Sunday, 6–9PM], [Monday...","{'Accessibility': None, 'Activities': None, 'A...",Closed ⋅ Opens 6PM Sun,None,https://www.google.com/maps/place//data=!4m2!3...
2,Freedom ElecTech,"Freedom ElecTech, 521 NC-105 Extension #3, Boo...",0x8850fa407ebe7c75:0x4e79ef72c51b2f64,None,36.210733,-81.666781,"[Electrician, Audio visual consultant, Audio v...",4.5,8,None,"[[Saturday, 10AM–2PM], [Sunday, Closed], [Mond...",None,Closed ⋅ Opens 8AM Mon,"[0x8850fa308494fc8d:0x4f7268d52953ea5c, 0x8850...",https://www.google.com/maps/place//data=!4m2!3...
3,Dietz Floral & Gifts,"Dietz Floral & Gifts, 549 Lincoln Ave, Bellevu...",0x8834f503bec584f3:0x4dcab098bbb36236,None,40.496279,-80.057246,"[Florist, Gift basket store, Gift shop]",4.3,38,None,"[[Saturday, 9AM–4PM], [Sunday, Closed], [Monda...",{'Accessibility': ['Wheelchair accessible entr...,Permanently closed,"[0x8834f3c5bb960cc3:0x8a96adaca50087f5, 0x8834...",https://www.google.com/maps/place//data=!4m2!3...
4,Beech Mountain Community Center,"Beech Mountain Community Center, 60 Flat Sprin...",0x88508c6d4a58520f:0x8bea3f3791d009b5,None,36.237188,-81.930071,[Auditorium],4.6,5,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x88508ce75fb3884b:0x820fd05a10d7602a, 0x8850...",https://www.google.com/maps/place//data=!4m2!3...


### Transformacion y extraccion de categorias "Hotels - Restaurants"

In [167]:
df['category'] = df['category'].astype(str).str.replace('[', '').str.replace(']', '')
df['category'] = df['category'].astype(str).str.replace("'", "")

In [168]:
filtro_restaurantes = df['category'].str.contains('restaurant', case=False, na=False)
filtro_hoteles = df['category'].str.contains('Hotels', case=False, na=False)

df = df[filtro_restaurantes | filtro_hoteles].copy()

### Top Estados "CA-FL-TX-NY-PA"

In [170]:
state_keywords = {'NY': 'New York', 'CA': 'California', 'TX': 'Texas', 'PA': 'Philadelphia', 'FL': 'Florida'}

def get_state(address):
    if pd.notna(address):
        
        for abbreviation, state in state_keywords.items():
            if state.lower() in address.lower():
                return state

        for abbreviation, state in state_keywords.items():
            if abbreviation.lower() in address.lower():
                return state

        for state in state_keywords.values():
            if state.lower() in address.lower():
                return state

    return 'Not address' 

df['states'] = df['address'].apply(get_state)

### Renombrar columnas - Eliminar 

In [171]:
df.drop(columns=['price'], axis=1, inplace=True)
df.drop(columns=['state'], axis=1, inplace=True)
df.drop(columns=['relative_results'], axis=1, inplace=True)

In [172]:
df.rename(columns={'MISC': 'attributes'}, inplace=True)

In [173]:
column_order = ['gmap_id', 'name', 'address','states','latitude','longitude','num_of_reviews','avg_rating','description','category','attributes','hours','url']
df = df[column_order]

### Transformacion datos 'Hours - Attributtes'

In [174]:
df['hours'] = df['hours'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
df['calendar'] = df['hours'].apply(lambda x: ', '.join([', '.join(map(str, item)) for item in x]) if isinstance(x, list) else x)
df.drop(columns=['hours'], axis=1, inplace=True)

In [175]:
df['attributes'] = df['attributes'].apply(str)
df['attributes'] = df['attributes'].astype('object')

In [176]:
def clean_attributes(attribute_str):

    cleaned_str = re.sub(r"[{}':,]", "", attribute_str)
    
    cleaned_str = re.sub(r"\barray\b|\[|\]|\(", "", cleaned_str)
    
    return cleaned_str

df['attributes'] = df['attributes'].apply(clean_attributes)

In [177]:
df.sample(5)

,gmap_id,name,address,states,latitude,longitude,num_of_reviews,avg_rating,description,category,attributes,url,calendar
228881,0x88e762e543b9ac13:0x2d2fde841ca3afc1,JR Grill & Market,"JR Grill & Market, 1 Jeff Fuqua Blvd MCO: Airs...",Florida,28.435524,-81.302449,47,2.1,Airport bar & grill with BBQ classics & other ...,Barbecue restaurant,Accessibility Wheelchair accessible entrance d...,https://www.google.com/maps/place//data=!4m2!3...,"Monday, 5AM–10PM, Tuesday, 5AM–10PM, Wednesday..."
221136,0x89e44cc7eac4d82d:0x55e381f0ab4e4021,Pal's Restaurant,"Pal's Restaurant, 43 Division St, East Greenwi...",Philadelphia,41.663891,-71.448136,164,4.1,None,Restaurant,Accessibility Wheelchair accessible entrance d...,https://www.google.com/maps/place//data=!4m2!3...,"Monday, 11:30AM–9PM, Tuesday, 11:30AM–9PM, Wed..."
237958,0x89b7b7d9891809c9:0xd5b87312d1bf6a41,Starbucks,"Starbucks, 1801 Columbia Rd NW, Washington, DC...",Not address,38.922716,-77.042921,326,4.0,Seattle-based coffeehouse chain known for its ...,Coffee shop Breakfast restaurant Cafe Coffee s...,Accessibility Wheelchair accessible entrance W...,https://www.google.com/maps/place//data=!4m2!3...,"Monday, 6AM–6PM, Tuesday, 6AM–6PM, Wednesday, ..."
80230,0x89c3954d0ee52873:0xf8917da6bedcd4e2,Hong Kong Garden,"Hong Kong Garden, 134 E Main St B, Somerville,...",Not address,40.566272,-74.606803,68,4.0,None,Chinese restaurant Delivery Restaurant,Accessibility Wheelchair accessible entrance d...,https://www.google.com/maps/place//data=!4m2!3...,"Wednesday, 11AM–9:30PM, Thursday, 11AM–9:30PM,..."
136006,0x89cb1592dbbd704b:0xa73e73f0daa564f6,Legends Caribbean Grill,"Legends Caribbean Grill, 1019 Ferndale Ave, Jo...",California,40.283786,-78.916243,26,4.6,None,Restaurant,Accessibility None Activities None Amenities G...,https://www.google.com/maps/place//data=!4m2!3...,"Tuesday, 10AM–9PM, Wednesday, 10AM–9PM, Thursd..."


In [178]:
df.to_parquet('11.parquet',index=False)